# Train Pytorch Explanation based Model for Fashion MNIST

# ref: tensorflow version from http://localhost:8888/notebooks/xai-adv/train_exp_based_model_v1.4_FMNIST_defender.ipynb

#### difference from FMNIST training is the number of epochs

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils import data as D
import os
import numpy as np
import numpy as np
from keras import datasets
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
from PIL import Image

Using TensorFlow backend.


In [2]:
dataset = 'mnist'
class_ind = 7

In [3]:
class Net(nn.Module):
    def __init__(self, features, num_classes, init_weights=True):
        super(Net, self).__init__()
        
        self.features = features
        self.classifier = nn.Sequential(
            nn.Linear(4*4*50, 500),
            nn.ReLU(True),
            nn.Linear(500, num_classes)
        )
        
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        
        # x are the logits values
        return x 
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        
        output = F.log_softmax(output, dim=1)
        
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            output = F.log_softmax(output, dim=1)
            
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [4]:
"""
torch.nn.Conv2d(in_channels, out_channels, kernel_size, 
stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
"""

def make_layers(cfg, in_channels, kernel_size, stride, padding, batch_norm=False):
    layers = []
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=kernel_size, padding=padding)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


"""
Refer VGG19_bn configurationh here: 
https://github.com/pytorch/vision/blob/76702a03d6cc2e4f431bfd1914d5e301c07bd489/torchvision/models/vgg.py#L63
"""
cfgs = {
    #'E': [64, 64, 'M',128, 128, 'M',256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M',512, 512, 512, 512, 'M'],
    'E': [20, 'M', 50, 'M']
}

model_layers = make_layers(cfgs['E'],in_channels=1, kernel_size=5, stride=1, padding=0, batch_norm=False)

In [5]:
model_layers

Sequential(
  (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [6]:
class CustomDS(D.Dataset):
    """
    A customized data loader.
    """
    def __init__(self, path, train=True):
        """ Intialize the dataset
        """
        if train:
            data_path = os.path.join(path,'x_train.npy')
            targets_path = os.path.join(path,'y_train.npy')
        else:
            data_path = os.path.join(path,'x_test.npy')
            targets_path = os.path.join(path,'y_test.npy')
        self.path = data_path
        self.data = np.load(data_path)
        self.targets = np.load(targets_path)
        #self.transform = transforms.ToTensor()
        self.transform = transforms.Compose([
                       transforms.ToTensor()
                       #transforms.Normalize((0.1307,), (0.3081,))
                   ])
        self.len = np.shape(self.data)[0]
        
    # You must override __getitem__ and __len__
    def __getitem__(self, index):
        """ Get a sample from the dataset
        """
        data = self.data[index]
        image = Image.fromarray(data)
        
        target = int(self.targets[index])
        
        #data = (data * 255).astype(np.uint8)
        #data = data.reshape(28,28)
        #image = Image.fromarray((data * 255).astype(np.uint8))
        #image = Image.fromarray(data.astype(np.uint8))
        
        return self.transform(image), target

    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.len

In [7]:
IS_DATA_READY = False

if not IS_DATA_READY:
    
    directory = 'data/defender/' + dataset + '/exp_model/for_target/' + str(class_ind)
    clean_intr_train_dir = 'data/defender/' + dataset + '/intr_of_clean_train_bytarget/' + str(class_ind)

    ox = np.load(clean_intr_train_dir + '/cleaned_intr_of_x_train.npy')
    ax = np.load(directory + '/intr_of_adv_with_target_'  + str(class_ind) + '.npy')

    print('orig {} : {}'.format(class_ind, ox.shape))
    print('adv {} : {}'.format(class_ind, ax.shape))
    print('\n')
    ax = np.vstack((ax,ax))
    ax = np.vstack((ax,ax))
    ax = np.vstack((ax,ax))
    ax = np.vstack((ax,ax))

    ax = ax[:ox.shape[0]]

    # shuffle ax
    indices = np.arange(ax.shape[0])
    np.random.shuffle(indices)
    ax = ax[indices]

    class_0 = ox
    class_1 = ax

    total_x = np.concatenate((class_0, class_1))
    total_x = total_x.reshape(-1,28,28)

    print('total shape {}'.format(total_x.shape))

    y_0 = np.zeros((len(class_0)))
    y_1 = np.ones((len(class_1)))

    total_y = np.concatenate((y_0, y_1))
    
    #train, test split
    x_train, x_test, y_train, y_test = train_test_split(total_x,total_y,test_size=0.20, random_state=42, shuffle=True)

    #train, valid split from the remaining train set (after test set has been taken out)
    x_train, x_valid, y_train, y_valid = train_test_split(x_train,y_train,test_size=0.20, random_state=42, shuffle=True)

    
    print('x_train shape : {}'.format(x_train.shape))
    print('y_train shape : {}'.format(y_train.shape))
    print('x_test shape : {}'.format(x_test.shape))
    print('y_test shape : {}'.format(y_test.shape))
    print('x_valid shape : {}'.format(x_valid.shape))
    print('y_valid shape : {}'.format(y_valid.shape))
    
    directory = 'data/defender/' + dataset + '/exp_model_data/for_target/' + str(class_ind)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    np.save(directory + '/x_train.npy', x_train)
    np.save(directory + '/y_train.npy', y_train)
    np.save(directory + '/x_test.npy', x_test)
    np.save(directory + '/y_test.npy', y_test)
    np.save(directory + '/x_valid.npy', x_valid)
    np.save(directory + '/y_valid.npy', y_valid)

else:
    directory = 'data/defender/' + dataset + '/exp_model_data/for_target/' + str(class_ind)
    
    x_train = np.load(directory + '/x_train.npy')
    y_train = np.load(directory + '/y_train.npy')
    x_test = np.load(directory + '/x_test.npy')
    y_test = np.load(directory + '/y_test.npy')
    x_valid = np.load(directory + '/x_valid.npy')
    y_valid = np.load(directory + '/x_valid.npy')
    
    print('x_train shape : {}'.format(x_train.shape))
    print('y_train shape : {}'.format(y_train.shape))
    print('x_test shape : {}'.format(x_test.shape))
    print('y_test shape : {}'.format(y_test.shape))
    print('x_valid shape : {}'.format(x_valid.shape))
    print('y_valid shape : {}'.format(y_valid.shape))  

orig 7 : (6238, 28, 28, 1)
adv 7 : (1256, 28, 28, 1)


total shape (12476, 28, 28)
x_train shape : (7984, 28, 28)
y_train shape : (7984,)
x_test shape : (2496, 28, 28)
y_test shape : (2496,)
x_valid shape : (1996, 28, 28)
y_valid shape : (1996,)


In [8]:
print(np.min(x_train[0]))
print(np.max(x_train[0]))

0.0
1.0


In [9]:
# Simple dataset. Only save path to image and load it and transform to tensor when call __getitem__.
filepath = directory
train_set = CustomDS(filepath, train=True)
test_set = CustomDS(filepath, train=False)

# total images in set
print(train_set.len)
print(test_set.len)

7984
2496


In [10]:
# main method
## Training settings
# input batch size for training (default: 64)
BATCH_SIZE = 64

# input batch size for testing (default: 1000)
TEST_BATCH_SIZE = 1000

# number of epochs to train
EPOCHS = 50

#learning rate (default: 0.01)
LR = 0.01

#SGD momentum (default: 0.5)
MOMENTUM = 0.5

# how many batches to wait before logging training status
LOG_INTERVAL = 10

SAVE_MODEL = True
SEED = 1
NO_CUDA = False
USE_CUDA = not NO_CUDA and torch.cuda.is_available()

NUM_CLASSES=2 # we are training a binary model (0 for normal or 1 for abnormal)

torch.manual_seed(SEED)

device = torch.device("cuda" if USE_CUDA else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if USE_CUDA else {}


In [11]:
model = Net(model_layers, num_classes=NUM_CLASSES).to(device)

In [12]:
model

Net(
  (features): Sequential(
    (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=800, out_features=500, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=500, out_features=2, bias=True)
  )
)

In [13]:
# Setup train and test loader

train_loader = D.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=False, **kwargs)
test_loader = D.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, **kwargs)


optimizer = optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM)

for epoch in range(1, EPOCHS + 1):
    train(model, device, train_loader, optimizer, epoch, LOG_INTERVAL)
    test(model, device, test_loader)

model_dir = directory + '/model'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
if (SAVE_MODEL):
    print('saving model to ', model_dir+"/exp_model.pt")
    torch.save(model.state_dict(),model_dir+"/exp_model.pt")

Train Epoch: 1 [0/7984 (0%)]	Loss: 0.693332
Train Epoch: 1 [640/7984 (8%)]	Loss: 0.693395
Train Epoch: 1 [1280/7984 (16%)]	Loss: 0.692422
Train Epoch: 1 [1920/7984 (24%)]	Loss: 0.691826
Train Epoch: 1 [2560/7984 (32%)]	Loss: 0.691399
Train Epoch: 1 [3200/7984 (40%)]	Loss: 0.691844
Train Epoch: 1 [3840/7984 (48%)]	Loss: 0.691812
Train Epoch: 1 [4480/7984 (56%)]	Loss: 0.690174
Train Epoch: 1 [5120/7984 (64%)]	Loss: 0.690628
Train Epoch: 1 [5760/7984 (72%)]	Loss: 0.690094
Train Epoch: 1 [6400/7984 (80%)]	Loss: 0.687307
Train Epoch: 1 [7040/7984 (88%)]	Loss: 0.689065
Train Epoch: 1 [7680/7984 (96%)]	Loss: 0.686036

Test set: Average loss: 0.6871, Accuracy: 2089/2496 (84%)

Train Epoch: 2 [0/7984 (0%)]	Loss: 0.686206
Train Epoch: 2 [640/7984 (8%)]	Loss: 0.687061
Train Epoch: 2 [1280/7984 (16%)]	Loss: 0.683027
Train Epoch: 2 [1920/7984 (24%)]	Loss: 0.684465
Train Epoch: 2 [2560/7984 (32%)]	Loss: 0.680394
Train Epoch: 2 [3200/7984 (40%)]	Loss: 0.684394
Train Epoch: 2 [3840/7984 (48%)]	Loss: 0

Train Epoch: 13 [1280/7984 (16%)]	Loss: 0.144561
Train Epoch: 13 [1920/7984 (24%)]	Loss: 0.140796
Train Epoch: 13 [2560/7984 (32%)]	Loss: 0.110862
Train Epoch: 13 [3200/7984 (40%)]	Loss: 0.175399
Train Epoch: 13 [3840/7984 (48%)]	Loss: 0.197115
Train Epoch: 13 [4480/7984 (56%)]	Loss: 0.157176
Train Epoch: 13 [5120/7984 (64%)]	Loss: 0.235650
Train Epoch: 13 [5760/7984 (72%)]	Loss: 0.275192
Train Epoch: 13 [6400/7984 (80%)]	Loss: 0.147930
Train Epoch: 13 [7040/7984 (88%)]	Loss: 0.091791
Train Epoch: 13 [7680/7984 (96%)]	Loss: 0.085589

Test set: Average loss: 0.1496, Accuracy: 2355/2496 (94%)

Train Epoch: 14 [0/7984 (0%)]	Loss: 0.088573
Train Epoch: 14 [640/7984 (8%)]	Loss: 0.189188
Train Epoch: 14 [1280/7984 (16%)]	Loss: 0.127428
Train Epoch: 14 [1920/7984 (24%)]	Loss: 0.127821
Train Epoch: 14 [2560/7984 (32%)]	Loss: 0.102737
Train Epoch: 14 [3200/7984 (40%)]	Loss: 0.154658
Train Epoch: 14 [3840/7984 (48%)]	Loss: 0.178185
Train Epoch: 14 [4480/7984 (56%)]	Loss: 0.148457
Train Epoch: 14

Train Epoch: 25 [0/7984 (0%)]	Loss: 0.024888
Train Epoch: 25 [640/7984 (8%)]	Loss: 0.106796
Train Epoch: 25 [1280/7984 (16%)]	Loss: 0.035740
Train Epoch: 25 [1920/7984 (24%)]	Loss: 0.050717
Train Epoch: 25 [2560/7984 (32%)]	Loss: 0.046701
Train Epoch: 25 [3200/7984 (40%)]	Loss: 0.047134
Train Epoch: 25 [3840/7984 (48%)]	Loss: 0.105653
Train Epoch: 25 [4480/7984 (56%)]	Loss: 0.099229
Train Epoch: 25 [5120/7984 (64%)]	Loss: 0.121766
Train Epoch: 25 [5760/7984 (72%)]	Loss: 0.109923
Train Epoch: 25 [6400/7984 (80%)]	Loss: 0.025353
Train Epoch: 25 [7040/7984 (88%)]	Loss: 0.035373
Train Epoch: 25 [7680/7984 (96%)]	Loss: 0.028391

Test set: Average loss: 0.0632, Accuracy: 2445/2496 (98%)

Train Epoch: 26 [0/7984 (0%)]	Loss: 0.022676
Train Epoch: 26 [640/7984 (8%)]	Loss: 0.106085
Train Epoch: 26 [1280/7984 (16%)]	Loss: 0.033874
Train Epoch: 26 [1920/7984 (24%)]	Loss: 0.047885
Train Epoch: 26 [2560/7984 (32%)]	Loss: 0.044411
Train Epoch: 26 [3200/7984 (40%)]	Loss: 0.044826
Train Epoch: 26 [3840


Test set: Average loss: 0.0424, Accuracy: 2461/2496 (99%)

Train Epoch: 37 [0/7984 (0%)]	Loss: 0.009018
Train Epoch: 37 [640/7984 (8%)]	Loss: 0.080590
Train Epoch: 37 [1280/7984 (16%)]	Loss: 0.019992
Train Epoch: 37 [1920/7984 (24%)]	Loss: 0.028414
Train Epoch: 37 [2560/7984 (32%)]	Loss: 0.023419
Train Epoch: 37 [3200/7984 (40%)]	Loss: 0.023807
Train Epoch: 37 [3840/7984 (48%)]	Loss: 0.062728
Train Epoch: 37 [4480/7984 (56%)]	Loss: 0.067538
Train Epoch: 37 [5120/7984 (64%)]	Loss: 0.085228
Train Epoch: 37 [5760/7984 (72%)]	Loss: 0.072581
Train Epoch: 37 [6400/7984 (80%)]	Loss: 0.007429
Train Epoch: 37 [7040/7984 (88%)]	Loss: 0.013988
Train Epoch: 37 [7680/7984 (96%)]	Loss: 0.011651

Test set: Average loss: 0.0413, Accuracy: 2461/2496 (99%)

Train Epoch: 38 [0/7984 (0%)]	Loss: 0.008445
Train Epoch: 38 [640/7984 (8%)]	Loss: 0.076520
Train Epoch: 38 [1280/7984 (16%)]	Loss: 0.019081
Train Epoch: 38 [1920/7984 (24%)]	Loss: 0.027521
Train Epoch: 38 [2560/7984 (32%)]	Loss: 0.021816
Train Epoc

Train Epoch: 48 [7040/7984 (88%)]	Loss: 0.006956
Train Epoch: 48 [7680/7984 (96%)]	Loss: 0.007441

Test set: Average loss: 0.0321, Accuracy: 2472/2496 (99%)

Train Epoch: 49 [0/7984 (0%)]	Loss: 0.004584
Train Epoch: 49 [640/7984 (8%)]	Loss: 0.045799
Train Epoch: 49 [1280/7984 (16%)]	Loss: 0.011768
Train Epoch: 49 [1920/7984 (24%)]	Loss: 0.018963
Train Epoch: 49 [2560/7984 (32%)]	Loss: 0.010179
Train Epoch: 49 [3200/7984 (40%)]	Loss: 0.012027
Train Epoch: 49 [3840/7984 (48%)]	Loss: 0.025120
Train Epoch: 49 [4480/7984 (56%)]	Loss: 0.050329
Train Epoch: 49 [5120/7984 (64%)]	Loss: 0.073038
Train Epoch: 49 [5760/7984 (72%)]	Loss: 0.054854
Train Epoch: 49 [6400/7984 (80%)]	Loss: 0.003062
Train Epoch: 49 [7040/7984 (88%)]	Loss: 0.006614
Train Epoch: 49 [7680/7984 (96%)]	Loss: 0.007374

Test set: Average loss: 0.0315, Accuracy: 2472/2496 (99%)

Train Epoch: 50 [0/7984 (0%)]	Loss: 0.004429
Train Epoch: 50 [640/7984 (8%)]	Loss: 0.043265
Train Epoch: 50 [1280/7984 (16%)]	Loss: 0.011198
Train Epoc